#### Imports

In [8]:
import os, json, math, time
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, random_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
torch.manual_seed(42); np.random.seed(42)
print("Device:", device)

Device: cuda


In [ ]:
from cnn import CNN1DEncoder, AstronetMVP
from dataset import ExoplanetDataset, collate

#### Data

to-do: 
get:
- global curves
- local curves
- labels
- ids
- tabular data

In [ ]:
# create data set
ds = ExoplanetDataset(
    global_curves = ...,
    local_curves = ...,
    labels = ...,
    ids = ...,
    tabular = ...
)

In [ ]:
# create train test split from data set
train_size = int(0.8 * len(ds))
test_size = len(ds) - train_size

ds_train, ds_test = random_split(ds, [train_size, test_size], generator = torch.Generator().manual_seed(42))

dl_train = DataLoader(ds_train, batch_size = 64, shuffle=True, collate_fn=collate, drop_last=False)
dl_test = DataLoader(ds_test, batch_size = 64, shuffle=True, collate_fn=collate, drop_last=False)


In [ ]:
# loss, optimizer, pos weight from train set
def compute_pos_weight_from_loader(dloader):
    pos = neg = 0
    with torch.no_grad():
        for _xg, _xl, _xt, _y, _ids in dloader:
            pos += (_y == 1).sum().item()
            neg += (_y == 0).sum().item()
    if pos == 0 or neg == 0: return None
    return torch.tensor([neg / max(pos, 1.0)], dtype=torch.float32, device=device)

# Infer tabular_dim from one train batch
xg0, xl0, xt0, y0, ids0 = next(iter(dl_train))
tabular_dim = 0 if xt0 is None else xt0.shape[1]

model = AstronetMVP(hidden=64, k_global=7, k_local=5, tabular_dim=tabular_dim).to(device)

pos_weight = compute_pos_weight_from_loader(dl_train)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight) if pos_weight is not None else nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)